In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
import acquire_plays_data
import prep_plays
import wrangle_plays_data
import re
plt.rc("figure", figsize=(12, 7))
plt.rc("font", size=14)

Acquire.py Loaded Successfully
Prep.py Loaded Successfully
Wrangle.py Loaded Successfully


In [2]:
df = acquire_plays_data.get_plays_data()

In [3]:
df = prep_plays.prep_plays_data()

In [4]:
X_train, y_train, X_validate, y_validate, X_test, y_test = wrangle_plays_data.train_validate_test(df)

In [5]:
from sklearn.preprocessing import StandardScaler, QuantileTransformer, PowerTransformer, RobustScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE, SelectKBest, f_regression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

In [6]:
X_train_scaled, X_validate_scaled, X_test_scaled = wrangle_plays_data.min_max_scale(X_train, X_validate, X_test)

In [7]:
def rfe_ranker(X_train_scaled, y_train, k):
    '''
    Uses Recursive Feature Elimination (RFE) to rank the given features in order of their usefulness in
    predicting a win with a linear regression model.
    '''
    # creating linear regression object
    lm = LinearRegression()

    # fitting linear regression model to features 
    lm.fit(X_train_scaled, y_train)

    # creating recursive feature elimination object and specifying to rank 5 of the best features
    rfe = RFE(lm, k)

    # using rfe object to transform features 
    x_rfe = rfe.fit_transform(X_train_scaled, y_train)

    feature_mask = rfe.support_

    # creating train df for rfe object 
    rfe_train = X_train_scaled

    # creating list of the top features per rfe
    rfe_features = rfe_train.loc[:,feature_mask].columns.tolist()

    # creating ranked list 
    feature_ranks = rfe.ranking_

    # creating list of feature names
    feature_names = rfe_train.columns.tolist()

    # create df that contains all features and their ranks
    rfe_ranks_df = pd.DataFrame({'Feature': feature_names, 'Rank': feature_ranks})

    # return df sorted by rank
    return rfe_ranks_df.sort_values('Rank')

In [8]:
X_train

,quarter,down,yardsToGo,team_by_comp_yds,defendersInTheBox,numberOfPassRushers,QB_under_pressure,absoluteYardlineNumber,epa,playResult,...,I_FORM,JUMBO,PISTOL,SHOTGUN,SINGLEBACK,WILDCAT,four_three,three_four,nickel,dime
4170,2,2,4,2,6.0,4.0,0.0,46.0,-0.611122,0,...,0,0,0,1,0,0,0,0,1,0
14076,4,1,10,12,6.0,5.0,0.0,65.0,-0.032445,6,...,0,0,0,0,0,0,0,1,0,0
15897,3,2,3,23,8.0,7.0,0.0,103.0,1.789159,7,...,0,0,0,1,0,0,0,0,1,0
2723,3,1,10,16,7.0,4.0,0.0,43.0,-0.025190,8,...,0,0,0,0,1,0,1,0,0,0
632,4,1,9,32,6.0,4.0,0.0,101.0,-0.066855,1,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2507,2,1,10,4,7.0,4.0,0.0,60.0,2.510391,42,...,0,0,0,0,1,0,0,0,1,0
10063,2,2,3,30,5.0,4.0,0.0,84.0,0.589207,6,...,0,0,0,1,0,0,0,0,0,1
10013,3,3,8,17,6.0,4.0,0.0,99.0,2.463213,11,...,0,0,0,0,0,0,0,0,0,1
3002,4,2,3,30,7.0,4.0,0.0,26.0,0.759128,8,...,0,0,0,0,1,0,0,0,1,0


In [9]:
rfe_ranker(X_train_scaled, y_train, k=8)

,Feature,Rank
2,yardsToGo,1
4,defendersInTheBox,1
5,numberOfPassRushers,1
6,QB_under_pressure,1
8,epa,1
9,DL,1
10,LB,1
11,DB,1
1,down,2
12,four_three,3


In [10]:
y_train

,pass_stopped
4170,1
14076,0
15897,0
2723,0
632,0
...,...
2507,0
10063,0
10013,0
3002,0


In [11]:
X_train_scaled = X_train_scaled[['yardsToGo','defendersInTheBox','numberOfPassRushers','QB_under_pressure','epa','DL','LB','DB']]
X_validate_scaled = X_validate_scaled[['yardsToGo','defendersInTheBox','numberOfPassRushers','QB_under_pressure','epa','DL','LB','DB']]
X_test_scaled = X_test_scaled[['yardsToGo','defendersInTheBox','numberOfPassRushers','QB_under_pressure','epa','DL','LB','DB']]

In [12]:
def knn(X_train_scaled, y_train, k):
    # KNN object
    knn = KNeighborsClassifier(n_neighbors=k, weights='uniform')

    # Fit the model
    knn = knn.fit(X_train_scaled, y_train)

    # Make predictions
    y_pred = knn.predict(X_train_scaled)

    # Estimate the probability
    y_pred_proba = knn.predict_proba(X_train_scaled)

    print('The k-neareast neighbor accuracy : {:.2f}\n'
                 .format(knn.score(X_train_scaled, y_train)))
    print(f'----------------------')
    # Confusion matrix
    #print(f'Confusion Matrix: \n\n {pd.crosstab(y_train, y_pred)}\n' )
    print(f'----------------------') 
    print("K-Nearest Neighbor Classification Report:\n", classification_report(y_train, y_pred))

    return knn

In [18]:
knn.score(X_train_scaled, y_train))

AttributeError: 'function' object has no attribute 'score'

In [13]:
print(X_train_scaled.shape)
print(y_train.shape)

(7140, 8)
(7140, 1)


In [23]:
knn(X_train_scaled, y_train, k=5)

The k-neareast neighbor accuracy : 0.87

----------------------
----------------------
K-Nearest Neighbor Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.88      0.90      4606
           1       0.79      0.85      0.82      2534

    accuracy                           0.87      7140
   macro avg       0.85      0.86      0.86      7140
weighted avg       0.87      0.87      0.87      7140



KNeighborsClassifier()

In [24]:
def knn_val(X_validate_scaled, y_validate, k):
    # KNN object
    knn = KNeighborsClassifier(n_neighbors=k, weights='uniform')
    
    # Fit the model
    knn = knn.fit(X_train_scaled, y_train)

    # Make predictions
    y_pred = knn.predict(X_validate_scaled)

    # Estimate the probability
    y_pred_proba = knn.predict_proba(X_validate_scaled)

    print('The k-neareast neighbor accuracy : {:.2f}\n'
                 .format(knn.score(X_validate_scaled, y_validate)))
    print(f'----------------------')
    # Confusion matrix
    #print(f'Confusion Matrix: \n\n {pd.crosstab(y_train, y_pred)}\n' )
    print(f'----------------------') 
    print("K-Nearest Neighbor Classification Report:\n", classification_report(y_validate, y_pred))

    return knn_val

In [25]:
knn_val(X_validate_scaled, y_validate, k=5)

The k-neareast neighbor accuracy : 0.80

----------------------
----------------------
K-Nearest Neighbor Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.84      0.85      3163
           1       0.69      0.74      0.71      1597

    accuracy                           0.80      4760
   macro avg       0.78      0.79      0.78      4760
weighted avg       0.81      0.80      0.80      4760



<function __main__.knn_val(X_validate_scaled, y_validate, k)>

In [30]:
def logistic_regression(X_train_scaled, y_train):
    '''
    This function takes in X_train (features using for model) and y_train (target 'win') and performs logistic
    regression giving us accuracy of the model and the classification report
    '''
    # Calling out funtion
    logit = LogisticRegression()

    # Fit the training data set
    logit = logit.fit(X_train_scaled, y_train)

    # Make predictions
    y_pred = logit.predict(X_train_scaled)

    #Accuracy of model
    score = logit.score(X_train_scaled, y_train)

    print(f'The logistic regression models accuracy is {round(score * 100,2)}%\n')
    print(f'----------------------')     
    #print(f'Confusion Matrix\n\n {pd.crosstab(y_train, y_pred)}\n') 
    print(f'----------------------') 
    
    # Coefficients for each feature  
    coef_df = pd.DataFrame(logit.coef_)

    # List comprehension for columns in X_train
    names = [column for column in X_train_scaled.columns]
    
    # Renaming columns to their names for coef_df
    coef_df.columns = names

    print(f'Classification Report\n {classification_report(y_train, y_pred)}')
    return coef_df, logit

In [31]:
from sklearn.linear_model import LogisticRegression, LinearRegression

In [32]:
coef_df, logit = logistic_regression(X_train_scaled, y_train)

The logistic regression models accuracy is 79.37%

----------------------
----------------------
Classification Report
               precision    recall  f1-score   support

           0       0.80      0.91      0.85      4606
           1       0.77      0.59      0.67      2534

    accuracy                           0.79      7140
   macro avg       0.79      0.75      0.76      7140
weighted avg       0.79      0.79      0.79      7140



In [35]:
def logistic_regression_val(X_validate_scaled, y_validate):
    '''
    This function takes in X_train (features using for model) and y_train (target 'win') and performs logistic
    regression giving us accuracy of the model and the classification report
    '''
    # Calling out funtion
    logit = LogisticRegression()

    # Fit the training data set
    logit = logit.fit(X_train_scaled, y_train)

    # Make predictions
    y_pred = logit.predict(X_validate_scaled)

    #Accuracy of model
    score = logit.score(X_validate_scaled, y_validate)

    print(f'The logistic regression models accuracy is {round(score * 100,2)}%\n')
    print(f'----------------------')     
    #print(f'Confusion Matrix\n\n {pd.crosstab(y_train, y_pred)}\n') 
    print(f'----------------------') 
    
    # Coefficients for each feature  
    coef_df = pd.DataFrame(logit.coef_)

    # List comprehension for columns in X_train
    names = [column for column in X_validate_scaled.columns]
    
    # Renaming columns to their names fovalidatef_df
    coef_df.columns = names

    print(f'Classification Report\n {classification_report(y_validate, y_pred)}')
    return coef_df, logit

In [36]:
coef_df, logit = logistic_regression_val(X_validate_scaled, y_validate)

The logistic regression models accuracy is 79.31%

----------------------
----------------------
Classification Report
               precision    recall  f1-score   support

           0       0.80      0.91      0.85      3163
           1       0.76      0.56      0.65      1597

    accuracy                           0.79      4760
   macro avg       0.78      0.74      0.75      4760
weighted avg       0.79      0.79      0.78      4760

